# 如何加载网页

本指南将介绍如何将网页加载到我们下游使用的 LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 格式中。网页包含文本、图像和其他多媒体元素，通常用 HTML 表示。它们可能包含指向其他页面或资源的链接。

LangChain 与许多适合网页的解析器集成。正确的解析器将取决于您的需求。下面我们演示两种可能性：

- [简单快速](/zh/docs/how_to/document_loader_web#simple-and-fast-text-extraction) 解析，我们将为每个网页恢复一个 `Document`，并将其内容表示为“展平”的字符串；
- [高级](/zh/docs/how_to/document_loader_web#advanced-parsing) 解析，我们将为每个网页恢复多个 `Document` 对象，允许用户识别和遍历章节、链接、表格和其他结构。

## 设置

对于“简单快速”的解析，我们将需要 `langchain-community` 和 `beautifulsoup4` 库：

In [ ]:
%pip install -qU langchain-community beautifulsoup4

对于高级解析，我们将使用 `langchain-unstructured`：

In [ ]:
%pip install -qU langchain-unstructured

## 简单快速的文本提取

如果您正在寻找嵌入在网页中的文本的简单字符串表示形式，以下方法很合适。它将返回一个 `Document` 对象列表——每页一个——其中包含页面文本的单个字符串。在底层，它使用了 `beautifulsoup4` Python 库。

LangChain 的文档加载器实现了 `lazy_load` 及其异步变体 `alazy_load`，它们返回 `Document` 对象的迭代器。我们将在下面使用它们。

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
print(f"{doc.metadata}\n")
print(doc.page_content[:500].strip())

{'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'A key feature of chatbots is their ability to use content of previous conversation turns as context. This state management can take several forms, including:', 'language': 'en'}

How to add memory to chatbots | ü¶úÔ∏èüîó LangChain







Skip to main contentShare your thoughts on AI agents. Take the 3-min survey.IntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1üí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingd


这基本上是页面 HTML 的文本转储。它可能包含标题和导航栏等无关信息。如果你熟悉预期的 HTML，可以通过 BeautifulSoup 指定所需的 `<div>` 类和其他参数。下面我们只解析文章的正文文本：

In [3]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="theme-doc-markdown markdown"),
    },
    bs_get_text_kwargs={"separator": " | ", "strip": True},
)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

In [4]:
print(f"{doc.metadata}\n")
print(doc.page_content[:500])

{'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/'}

How to add memory to chatbots | A key feature of chatbots is their ability to use content of previous conversation turns as context. This state management can take several forms, including: | Simply stuffing previous messages into a chat model prompt. | The above, but trimming old messages to reduce the amount of distracting information the model has to deal with. | More complex modifications like synthesizing summaries for long running conversations. | We'll go into more detail on a few techniq


In [5]:
print(doc.page_content[-500:])

a greeting. Nemo then asks the AI how it is doing, and the AI responds that it is fine.'), | HumanMessage(content='What did I say my name was?'), | AIMessage(content='You introduced yourself as Nemo. How can I assist you today, Nemo?')] | Note that invoking the chain again will generate another summary generated from the initial summary plus new messages and so on. You could also design a hybrid approach where a certain number of messages are retained in chat history while others are summarized.


请注意，这需要对正文在底层 HTML 中的表示方式具有预先的技术知识。

我们可以通过各种设置来参数化 `WebBaseLoader`，从而指定请求头、速率限制以及 `BeautifulSoup` 的解析器和其他 kwargs。请参阅其 [API 参考](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)了解详情。

## 高级解析

如果我们希望对页面内容进行更细粒度的控制或处理，则此方法非常合适。下面，我们不为每个页面生成一个 `Document` 并通过 BeautifulSoup 控制其内容，而是生成多个代表页面上不同结构的 `Document` 对象。这些结构可以包括节标题及其对应的正文文本、列表或枚举、表格等。

它底层使用了 `langchain-unstructured` 库。有关将 [Unstructured](https://docs.unstructured.io/welcome) 与 LangChain 结合使用的更多信息，请参阅 [集成文档](/docs/integrations/document_loaders/unstructured_file/)。

In [6]:
from langchain_unstructured import UnstructuredLoader

page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"
loader = UnstructuredLoader(web_url=page_url)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

INFO: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: NumExpr defaulting to 8 threads.


请注意，在不预先了解页面 HTML 结构的情况下，我们可以恢复正文内容的自然组织：

In [7]:
for doc in docs[:5]:
    print(doc.page_content)

How to add memory to chatbots
A key feature of chatbots is their ability to use content of previous conversation turns as context. This state management can take several forms, including:
Simply stuffing previous messages into a chat model prompt.
The above, but trimming old messages to reduce the amount of distracting information the model has to deal with.
More complex modifications like synthesizing summaries for long running conversations.
ERROR! Session/line number was not unique in database. History logging moved to new session 2747


### 从特定区域提取内容

每个 `Document` 对象代表页面的一个元素。其元数据包含有用的信息，例如其类别：

In [8]:
for doc in docs[:5]:
    print(f'{doc.metadata["category"]}: {doc.page_content}')

Title: How to add memory to chatbots
NarrativeText: A key feature of chatbots is their ability to use content of previous conversation turns as context. This state management can take several forms, including:
ListItem: Simply stuffing previous messages into a chat model prompt.
ListItem: The above, but trimming old messages to reduce the amount of distracting information the model has to deal with.
ListItem: More complex modifications like synthesizing summaries for long running conversations.


元素也可能具有父子关系——例如，一段文字可能属于一个带有标题的部分。如果一个部分特别引人关注（例如，用于索引），我们可以隔离相应的 `Document` 对象。

例如，下面我们加载两个网页的“设置”部分的内容：

In [9]:
from typing import List

from langchain_core.documents import Document


async def _get_setup_docs_from_url(url: str) -> List[Document]:
    loader = UnstructuredLoader(web_url=url)

    setup_docs = []
    parent_id = -1
    async for doc in loader.alazy_load():
        if doc.metadata["category"] == "Title" and doc.page_content.startswith("Setup"):
            parent_id = doc.metadata["element_id"]
        if doc.metadata.get("parent_id") == parent_id:
            setup_docs.append(doc)

    return setup_docs


page_urls = [
    "https://python.langchain.com/docs/how_to/chatbots_memory/",
    "https://python.langchain.com/docs/how_to/chatbots_tools/",
]
setup_docs = []
for url in page_urls:
    page_setup_docs = await _get_setup_docs_from_url(url)
    setup_docs.extend(page_setup_docs)

In [10]:
from collections import defaultdict

setup_text = defaultdict(str)

for doc in setup_docs:
    url = doc.metadata["url"]
    setup_text[url] += f"{doc.page_content}\n"

dict(setup_text)

{'https://python.langchain.com/docs/how_to/chatbots_memory/': "You'll need to install a few packages, and have your OpenAI API key set as an environment variable named OPENAI_API_KEY:\n%pip install --upgrade --quiet langchain langchain-openai\n\n# Set env var OPENAI_API_KEY or load from a .env file:\nimport dotenv\n\ndotenv.load_dotenv()\nWARNING: You are using pip version 22.0.4; however, version 23.3.2 is available.\nYou should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.\nNote: you may need to restart the kernel to use updated packages.\n",
 'https://python.langchain.com/docs/how_to/chatbots_tools/': "For this guide, we'll be using a tool calling agent with a single tool for searching the web. The default will be powered by Tavily, but you can switch it out for any similar tool. The rest of this section will assume you're using Tavily.\nYou'll need to sign up for an account on the Tavily website, and install the

### 对页面内容进行向量搜索

将页面内容加载到 LangChain 的 `Document` 对象后，我们就可以像平常一样对它们进行索引（例如，用于 RAG 应用）。下面我们使用 OpenAI [embedding](/docs/concepts/embedding_models) 模型，但任何 LangChain embedding 模型都可以。

In [ ]:
%pip install -qU langchain-openai

In [11]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(setup_docs, OpenAIEmbeddings())
retrieved_docs = vector_store.similarity_search("Install Tavily", k=2)
for doc in retrieved_docs:
    print(f'Page {doc.metadata["url"]}: {doc.page_content[:300]}\n')

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Page https://python.langchain.com/docs/how_to/chatbots_tools/: You'll need to sign up for an account on the Tavily website, and install the following packages:

Page https://python.langchain.com/docs/how_to/chatbots_tools/: For this guide, we'll be using a tool calling agent with a single tool for searching the web. The default will be powered by Tavily, but you can switch it out for any similar tool. The rest of this section will assume you're using Tavily.



## 其他网页加载器

有关可用的 LangChain 网页加载器列表，请参阅[此表](/docs/integrations/document_loaders/#webpages)。